https://www.machinecurve.com/index.php/2019/12/30/how-to-create-a-variational-autoencoder-with-keras/

In [52]:
import math
import numpy as np

In [53]:
Training_Images = np.load('/fast/gmooers/Preprocessed_Data/W_Trial/Space_Time_W_Training.npy')
Test_Images = np.load('/fast/gmooers/Preprocessed_Data/W_Trial/Space_Time_W_Test.npy')

In [54]:
import keras
from keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Lambda, Reshape
from keras.layers import BatchNormalization
from keras.models import Model
from keras.datasets import mnist
from keras.losses import binary_crossentropy
from keras import backend as K
import matplotlib.pyplot as plt

In [55]:
print(Training_Images.shape)

(8371, 30, 128)


In [56]:
batch_size = 128
no_epochs = 10
verbosity = 1
latent_dim = 2
num_channels = 1
input_shape = (len(Training_Images[0]), len(Training_Images[0][0]),1)
img_width = len(Training_Images[0])
img_height = len(Training_Images[0][0])

In [57]:
#Encoder
i = Input(shape=input_shape, name='encoder_input')
cx = Conv2D(filters=8, kernel_size=3, strides=2, padding='same', activation='relu')(i)
cx = BatchNormalization()(cx)
cx = Conv2D(filters=16, kernel_size=3, strides=2, padding='same', activation='relu')(cx)
cx = BatchNormalization()(cx)
x = Flatten()(cx)
x = Dense(20, activation='relu')(x)
x = BatchNormalization()(x)
mu = Dense(latent_dim, name='latent_mu')(x)
sigma = Dense(latent_dim, name='latent_sigma')(x)

In [58]:
conv_shape = K.int_shape(cx)

In [59]:
# Define sampling with reparameterization trick
def sample_z(args):
    mu, sigma = args
    batch = K.shape(mu)[0]
    dim = K.int_shape(mu)[1]
    eps = K.random_normal(shape=(batch, dim))
    return mu + K.exp(sigma / 2) * eps

In [60]:
# Use reparameterization trick to ensure correct gradient
z = Lambda(sample_z, output_shape=(latent_dim, ), name='z')([mu, sigma])

In [61]:
# Instantiate encoder
encoder = Model(i, [mu, sigma, z], name='encoder')
encoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 30, 128, 1)   0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 15, 64, 8)    80          encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization_7 (BatchNor (None, 15, 64, 8)    32          conv2d_3[0][0]                   
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 8, 32, 16)    1168        batch_normalization_7[0][0]      
__________________________________________________________________________________________________
batch_norm

In [62]:
# =================
# Decoder
# =================

# Definition
d_i = Input(shape=(latent_dim, ), name='decoder_input')
x = Dense(conv_shape[1] * conv_shape[2] * conv_shape[3], activation='relu')(d_i)
x = BatchNormalization()(x)
x = Reshape((conv_shape[1], conv_shape[2], conv_shape[3]))(x)
cx = Conv2DTranspose(filters=16, kernel_size=3, strides=2, padding='same', activation='relu')(x)
cx = BatchNormalization()(cx)
cx = Conv2DTranspose(filters=8, kernel_size=3, strides=2, padding='same',  activation='relu')(cx)
cx = BatchNormalization()(cx)
o = Conv2DTranspose(filters=num_channels, kernel_size=3, activation='sigmoid', padding='same', name='decoder_output')(cx)

In [63]:
# Instantiate decoder
decoder = Model(d_i, o, name='decoder')
decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 2)                 0         
_________________________________________________________________
dense_25 (Dense)             (None, 4096)              12288     
_________________________________________________________________
batch_normalization_10 (Batc (None, 4096)              16384     
_________________________________________________________________
reshape_5 (Reshape)          (None, 8, 32, 16)         0         
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 16, 64, 16)        2320      
_________________________________________________________________
batch_normalization_11 (Batc (None, 16, 64, 16)        64        
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 32, 128, 8)        1160      
__________

In [64]:
# =================
# VAE as a whole
# =================

# Instantiate VAE
vae_outputs = decoder(encoder(i)[2])
vae         = Model(i, vae_outputs, name='vae')
vae.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 30, 128, 1)        0         
_________________________________________________________________
encoder (Model)              [(None, 2), (None, 2), (N 83448     
_________________________________________________________________
decoder (Model)              (None, 32, 128, 1)        32321     
Total params: 115,769
Trainable params: 107,441
Non-trainable params: 8,328
_________________________________________________________________


In [65]:
# Define loss
def kl_reconstruction_loss(true, pred):
    # Reconstruction loss
    reconstruction_loss = binary_crossentropy(K.flatten(true), K.flatten(pred)) * img_width * img_height
    # KL divergence loss
    kl_loss = 1 + sigma - K.square(mu) - K.exp(sigma)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    # Total loss = 50% rec + 50% KL divergence loss
    return K.mean(reconstruction_loss + kl_loss)

In [66]:
# Compile VAE
vae.compile(optimizer='adam', loss=kl_reconstruction_loss)

In [68]:
Training_Images = Training_Images.reshape(Training_Images.shape+(1,))
Test_Images = Test_Images.reshape(Test_Images.shape+(1,))

In [72]:
# Train autoencoder
validation_split = 0.2
h =vae.fit(Training_Images, Training_Images, epochs = no_epochs, batch_size = batch_size, validation_split = validation_split)

Train on 6696 samples, validate on 1675 samples
Epoch 1/10


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node encoder_1/conv2d_3/convolution}}]]
	 [[{{node loss_4/mul}}]]